# Linear Regression project - Football matches

Using the Matches csv file from [Club-Football-Match-Data-2000-2025](https://github.com/xgabora/Club-Football-Match-Data-2000-2025) to build a linear regression model which predicts the score of a match given some factors.

In [2]:
import pandas as pd

In [26]:
dtypes = {'MatchTime':"str"}
parse_dates = ['MatchDate']
df = pd.read_csv('Matches.csv', dtype=dtypes, parse_dates=parse_dates)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230557 entries, 0 to 230556
Data columns (total 48 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Division     230557 non-null  object        
 1   MatchDate    230557 non-null  datetime64[ns]
 2   MatchTime    99072 non-null   object        
 3   HomeTeam     230557 non-null  object        
 4   AwayTeam     230557 non-null  object        
 5   HomeElo      141597 non-null  float64       
 6   AwayElo      141528 non-null  float64       
 7   Form3Home    229057 non-null  float64       
 8   Form5Home    229057 non-null  float64       
 9   Form3Away    229057 non-null  float64       
 10  Form5Away    229057 non-null  float64       
 11  FTHome       230554 non-null  float64       
 12  FTAway       230554 non-null  float64       
 13  FTResult     230554 non-null  object        
 14  HTHome       175977 non-null  float64       
 15  HTAway       175977 non-null  floa